In [1]:
import pandas as pd

melb_data_ps=pd.read_csv('data/melb_data_ps.csv', sep=',')
print(melb_data_ps['Address'].nunique()) # этот метод отображает кол-во НЕ уникальных элементов в столбце
# 13378 - это кол-во неуник элементов. Они мешают правильному анализу, даже вредят, но чтобы правильно их убрать, необходимо разобраться что они из себя представляют:
print(melb_data_ps['Address'].loc[177])
print(melb_data_ps['Address'].loc[5680])
print(melb_data_ps['Address'].loc[10522])
# из этого понимаем что адресс строиться слудующим образом. Сначала дом, корпус, название улицы и далее подтип улицы, а так же географическая отметка, которая нам портит всю картину

# На вход данной функции поступает строка с адресом.
def get_street_type(address): # создали фун-ю чтобы удалить георграфические отметки
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E'] # создали список включающий в себя метки, которые хотим убрать
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list: # проверяем если элемент не соответ-ет географ-кой метке, то 
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2] # если есть географическая метка, то возвращаем второй элемент с конца 
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type #если нет географической метки, то берем последний элемент с конца

# теперь прогоняем через фун-ю наш столбец с адресами:
street_types=melb_data_ps['Address'].apply(get_street_type) #обрати внимание, объявленная переменная не одинаковая как в предыдущей фун-ии!
display(street_types) # в итоге появился объект Series, в котором элемент это подтип улицы и все

# !!! ВАЖНО: функция была написана только для одного элемента, а метод apply ко всему столбцу !!!
print(street_types.nunique()) # теперь видим что уникальных значений осталось 56

# теперь посмотрим на разбивку(частота появления каждого из уникальных значений)
print(street_types.value_counts())

#продолжаем улучшать результат по уникальным элементам:
popular_street_types=street_types.value_counts().nlargest(10).index # сделали выборку по 10ти самым популярным элементам в результате который записан в value_counts(от самого популярного к наименее популярному)
print(popular_street_types)
# Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, 
# lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. 
# Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:
melb_data_ps['StreetType']=street_types.apply(lambda x: x if x in popular_street_types else 'other')
display(melb_data_ps['StreetType'])
# ещё раз глянем на кол-во уникальных элементов в таблице:
print(melb_data_ps['StreetType'].nunique()) # 11
# теперь нам признак Address не нужен, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude
melb_data_ps=melb_data_ps.drop('Address', axis=1) # дропнем столбец Address

# Practice. из задания 3.3 возвращаем weekdaysale + приводим к daytime столбец Time
melb_data_ps['Date']=pd.to_datetime(melb_data_ps['Date'], dayfirst=True)
melb_data_ps['weekdaysale']=melb_data_ps['Date'].dt.dayofweek
display(melb_data_ps['weekdaysale'])

# Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае
def get_weekend(weekday):
    if weekday==5 or weekday==6:
        return 1
    else:
        return 0
melb_data_ps['Weekend']=melb_data_ps['weekdaysale'].apply(get_weekend) # создал признак weekend в основном сете на основе столбца weekdaysale, прогнанного через фун-ю выходного дня 
print(round(melb_data_ps[melb_data_ps['Weekend']==1]['Price'].mean(), 2))

# Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом:
# оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
# Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, 
# проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

uniccomp=melb_data_ps['SellerG'].value_counts().nlargest(49).index # сделал выборку по 49 самым популярным компаниям
melb_data_ps['SellerG']=melb_data_ps['SellerG'].apply(lambda x: x if x in uniccomp else "other") # преобразил столбец на основе 49 самых популярных компаний, остальные назвал "other"
display(melb_data_ps['SellerG']) # its work ) просто проверил
priceforother=melb_data_ps[melb_data_ps['SellerG']=='other']['Price'].min() # вывели мин по other
priceforNelson=melb_data_ps[melb_data_ps['SellerG']=='Nelson']['Price'].min() # вывели по Nelson
print(round((priceforNelson/priceforother), 1)) # ответ 1.3

# задание в платформе SF
# Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов.
# В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. 
# Структура текста в столбце фиксирована и не может измениться.
# Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. 
# Не забудьте привести результат к целому числу.
# Примечание. Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. Учтите это при построении своей функции.

import pandas as pd

def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца'] # задали переменную со списком месяц
    year_key_words = ['год', 'лет', 'года'] # задали переменную со списком год
    args_splited = arg.split(' ') # задали переменную которая делит аргументы через пробелы
    month = 0 # обозначили месяц
    year = 0 # обозначили год
    for i in range(len(args_splited)): # для каждого i в строчке/списке делим согласно нашей переменной 
        if args_splited[i] in month_key_words: # если в строчке есть элемент, похожий на те, которые есть в списке месяц то
            month = args_splited[i-1] # месяц теперь равен элементу, который идет как предыдуший от этого слова
        if args_splited[i] in year_key_words: # если в строчке етсь элемент, похожий на те, которые есть в списке год то
            year = args_splited[i-1] # год теперь равен элементу, который идет как предыдуший от этого слова
    return int(year)*12 + int(month) # возвращаем в месяцах опыт работы

13378
2/119 Railway St N
39 Hardy St
81 Parsons St


0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

56
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso  

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

11


0        5
1        3
2        5
3        5
4        5
        ..
13575    5
13576    5
13577    5
13578    5
13579    5
Name: weekdaysale, Length: 13580, dtype: int64

1081198.64


0          Biggin
1          Biggin
2          Biggin
3          Biggin
4          Nelson
           ...   
13575       Barry
13576    Williams
13577       Raine
13578     Sweeney
13579     Village
Name: SellerG, Length: 13580, dtype: object

1.3
